# ADM - HW 3

# 1. Data Collection

## 1.1 Get the list of master's degree courses

In [ ]:
# se to true to install bs4 & selenium
if False: 
    pip install bs4
    pip install selenium

In [1]:
from bs4 import BeautifulSoup
import requests
import os 
import itertools
import time
import pandas as pd 
import csv 

First of all, we get a list of all the master's degree courses we need.

In [2]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class':'courseLink'})
  result_list=[]
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

We notice that the url of th various pages is *https://www.findamasters.com/masters-degrees/msc-degrees/?PG=* followed by the number of the page in the MSc courses catalogue we're in. 

In [3]:
links = []
for page in range(1, 401):
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='+str(page)
    links += extract_masters(url)
    print(page, len(links))

1 15
2 30
3 45
4 60
5 75
6 90
7 105
8 120
9 135
10 150
11 165
12 180
13 195
14 210
15 225
16 240
17 255
18 270
19 285
20 300
21 315
22 330
23 345
24 360
25 375
26 390
27 405
28 420
29 435
30 450
31 465
32 480
33 495
34 510
35 525
36 540
37 555
38 570
39 585
40 600
41 615
42 630
43 645
44 660
45 675
46 690
47 705
48 720
49 735
50 750
51 765
52 780
53 795
54 810
55 825
56 840
57 855
58 870
59 885
60 900
61 915
62 930
63 945
64 960
65 975
66 990
67 1005
68 1020
69 1035
70 1050
71 1065
72 1080
73 1095
74 1110
75 1125
76 1140
77 1155
78 1170
79 1185
80 1200
81 1215
82 1230
83 1245
84 1260
85 1275
86 1290
87 1305
88 1320
89 1335
90 1350
91 1365
92 1380
93 1395
94 1410
95 1425
96 1440
97 1455
98 1470
99 1485
100 1500
101 1515
102 1530
103 1545
104 1560
105 1575
106 1590
107 1605
108 1620
109 1635
110 1650
111 1665
112 1680
113 1695
114 1710
115 1725
116 1740
117 1755
118 1770
119 1785
120 1800
121 1815
122 1830
123 1845
124 1860
125 1875
126 1890
127 1905
128 1920
129 1935
130 1950
131 1965
1

We want to verify we retrieved all of the needed urls, so we check for the length of the list

In [5]:
print(len(links))

6000


It's the expected length, so we can create and write the required *.txt* file.   

In [4]:
file = open('msc_links.txt','w')
for link in links:
	file.write(link[0]+'\n')

file.close()

Now, if we look at the file it's written in the correct way (:

## 1.2 Crawl master's degree pages 

In [62]:
def download_html(url, dest_folder):
    #path is the local path we're working in 
    if not os.path.exists(dest_folder):  # create folder if it does not exist
        os.makedirs(dest_folder) 
    
    filename = url.replace('https://www.findamasters.com//masters-degrees/course/', '').replace('/', '_')
    file_path = os.path.join(dest_folder, filename)
    url_req = requests.get(url)

    if url_req.ok:
        soup = BeautifulSoup(url_req.text, 'html.parser')
        with open(file_path, 'w') as file: 
            file.write(soup.prettify())
        
        file.close()
        

In [68]:
### SAVING HTML & ORGANIZING THEM IN FOLDERS ###
file = open('msc_links.txt', 'r')
lines_list = file.readlines()
file.close()

In [73]:
for i in range(1, 401):
    folder_i = 'page_' + str(i)
        
    #selecting the lines corresponding to the i-th page
    start = (i-1)*15
    stop =  i*15
    needed_lines = lines_list[start : stop]

    #saving the corresponding html in "page_i" folder
    for line in needed_lines:
        url  = 'https://www.findamasters.com/' + str(line)[:-1]
        download_html(str(url), folder_i)
        time.sleep(2)
    

## 1.3 Parse downloaded pages

Creating a DataSet starting from the scrapped *.tsv* files. 

In [2]:
msc_degrees = pd.DataFrame(columns=['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url'])
for i in range(1, 6001): 
    course_file = csv.reader(open(os.path.join('courses_tsv', 'course_'+str(i))+'.tsv', 'r'), delimiter='\t')
    line = pd.DataFrame(course_file, columns=['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url'], index = [i])
    msc_degrees = pd.concat([msc_degrees, line])


To see if we created the DataFrame in the correct way 

In [3]:
msc_degrees

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
1,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MA MSc Other Pre-Masters Masters Module,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
4,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,Master of Science in Business (MSB) Our Mas...,See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...
5997,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,Material culture and artefact studies combine...,September,Please see the university website for further...,MSc PGDip,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5998,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,Harbour.Space’s Master's of Computer Science ...,"September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
5999,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme ...,"September, January","€29,900/year",MBA MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...


# 2. Search Engine

## 2.0 Preprocessing - preprocessing the text

In [4]:
import nltk

We are required to preprocess all the informations retrieved for the courses, but since loosing punctuation in the url will lead us to loose its functionality, we won't preprocess that column (also, the info contained in the url aren't textual info). Moreover, we won't preprocess the fees column since there's a particoular preprocessing required for that column.

#### Punctuation & stopwords 

In [6]:
nltk.download('punkt') 
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/camillabrigandi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/camillabrigandi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stopw = stopwords.words('english')

AttributeError: 'WordListCorpusReader' object has no attribute 'punkt'

In [15]:
stem_columns=['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'modality', 'duration', 'city', 'country', 'administration']

msc_stem = msc_degrees.copy()
#preprocessing 'descritpion' column: removing punctuation and stopwords
msc_stem['description'] = msc_degrees.description.apply(lambda row: nltk.word_tokenize(row)).apply(lambda row: [word for word in row if word.isalnum() and not word in stopw])

# preprocessing 'descritpion' column: stemming
msc_stem['description'] = msc_degrees.description.apply(lambda row: [nltk.PorterStemmer().stem(word) for word in row])

In [16]:
msc_stem

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
1,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,"[ , p, r, o, g, r, e, s, s, , y, o, u, r, , ...","January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,"[ , p, r, o, g, r, a, m, m, e, , o, v, e, r, ...",September,18.000 €,MSc,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,"[ , e, m, b, a, r, k, , o, n, , a, , p, r, ...",September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
4,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,"[ , t, h, e, , a, n, a, l, y, t, i, c, a, l, ...",See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,"[ , b, u, s, i, n, e, s, s, e, s, , a, n, d, ...",September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,"[ , m, a, s, t, e, r, , o, f, , s, c, i, e, ...",See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...
5997,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,"[ , m, a, t, e, r, i, a, l, , c, u, l, t, u, ...",September,Please see the university website for further...,MSc,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5998,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,"[ , h, a, r, b, o, u, r, ., s, p, a, c, e, ’, ...","September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
5999,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,"[ , h, a, r, b, o, u, r, ., s, p, a, c, e, ', ...","September, January","€29,900/year",MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...


In [17]:
msc_degrees

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
1,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MSc,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
4,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,Master of Science in Business (MSB) Our Mas...,See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...
5997,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,Material culture and artefact studies combine...,September,Please see the university website for further...,MSc,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5998,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,Harbour.Space’s Master's of Computer Science ...,"September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
5999,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme ...,"September, January","€29,900/year",MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
